In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from init_objects import *
from hft_model import *

# Baseline simulation

Simulate the baseline model dynamics to produce data on:

1. Autocorrelation returns
2. Autocorrelation abs returns
3. Excess Kurtosis
4. Volume / Volatility correlation
5. HFT Sharpe ratio 
6. Percentage trades by HFT 

In [8]:
NRUNS = 1

In [9]:
parameters = {
    # global parameters
    "n_lft": 20,
    "n_hft": 20,
    "ticks": 600, # 390 ticks per day
    "fundamental_value": 100,
    "lft_sample_size": 0.1,
    "hft_sample_size": 0.1,
    "std_noise": 0.01,
    "std_LFT_price": 0.01,
    "std_LFT_vol": 4,
    "std_HFT_vol": 4,
    "return_on_investment": 0.5,
    "investment_frequency": 5000,
    "max_order_expiration_ticks": 30,
    # lft parameters
    "w_fundamentalists": 2.5, #1
    "w_chartists": 1.6, # 10
    "w_random": 2.0,
    "spread_max": 0.25,
    # hft parameters
    "inventory_target": 1,
    "minimum_price_increment": 0.001,
    "investment_fraction": 0.00,
    "hfm_risk_aversion": 25,
    "hfm_fixed_vol": 1,
    # initial values
    "wealth": 300,
    "horizon_min": 1,
    "horizon_max": 4,
    "av_return_interval_max": 4,
    "init_price": 1,
    "agent_order_price_variability": (1, 1),
    "total_hft_money": 2000,
    "hft_speed": 1,
    "hft_init_investment": 0
}

## Run model

In [10]:
# 1 initalise model objects
high_frequency_traders, low_frequency_traders, orderbook = init_objects(parameters)

In [11]:
hfms = []
lfts = []
orderbooks = []
for seed in range(NRUNS): # optional, remove tqdm(), it is used for the progress bar
    hft, lft, ob = hft_model(high_frequency_traders, low_frequency_traders, orderbook, parameters, seed)
    hfms.append(hft)
    lfts.append(lft)
    orderbooks.append(ob)

19.8621049972
14.3696126634
14.3706126634
34.7910926378
3.25836148658
19.8641049972
34.7940926378
19.8661049972
2.37930575883
2.38030575883
2.38130575883
2.38230575883
2.38330575883
0.970024207247
0.970024207247
0.970024207247
0.583943090189
1.80671183603
0.585943090189
7.98741129838
7.98841129838
7.38099931196
49.4834794559
49.4844794559
22.9691066038
22.9691066038
22.9701066038
1.54850963663
37.4265796027
50.1763443743
50.1763443743
50.1773443743
1.19321346043
1.19321346043
1.19421346043
1.19521346043
1.19621346043
-15.9253144954
1.92655763105
15.8102760828
15.8122760828
1.14560527024
1.14660527024
47.5165071404
0.189705856206
0.190705856206
0.191705856206
0.192705856206
2.52843782832
2.52943782832
2.53043782832
11.6617401782
11.6617401782
11.6627401782
11.6637401782
11.4862819896
11.4862819896
11.4872819896
11.4882819896
5.80286271411
5.80286271411
-40.6682328404
5.80486271411
9.58231275951
9.58231275951
10.3960301816
-35.4116154282
10.3980301816
7.70781544552
3.61557656586
3.615576

## Retrieve data

In [12]:
average_tick_price = []
total_tick_volume = []
returns = []
autocorr_returns = []
absolute_returns = []
autocorr_abs_returns = []
for idx, orderbook in enumerate(orderbooks):
    average_tick_price.append([np.mean(prices) for prices in orderbook.transaction_prices_history])
    total_tick_volume.append([sum(volumes) for volumes in orderbook.transaction_volumes_history])
    returns.append(pd.Series(np.array(average_tick_price[idx])).pct_change())
    autocorr_returns.append([pd.Series(returns[idx]).autocorr(lag=lag) for lag in range(25)])
    absolute_returns.append(pd.Series(returns[idx]).abs())
    autocorr_abs_returns.append([absolute_returns[idx].autocorr(lag=lag) for lag in range(25)])